# 이미지파일 로딩

In [1]:
import imageio
import torch

In [2]:
img_arr=imageio.imread('data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

C:\Users\Eunji\AppData\Local\Temp\ipykernel_11320\2715363302.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr=imageio.imread('data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

In [3]:
img=torch.from_numpy(img_arr)
out=img.permute(2,0,1)

In [4]:
batch_size=3
batch= torch.zeros(batch_size,3,256,256,dtype=torch.uint8)

In [5]:
import os

data_dir='data/p1ch4/image-cats/'
filenames=[name for name in os.listdir(data_dir)
          if os.path.splitext(name)[-1]=='.png']

for i,filename in enumerate(filenames):
    img_arr=imageio.imread(os.path.join(data_dir,filename))
    img_t=torch.from_numpy(img_arr)
    img_t=img_t.permute(2,0,1)
    img_t=img_t[:3]
    batch[i]=img_t


C:\Users\Eunji\AppData\Local\Temp\ipykernel_11320\2139627687.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr=imageio.imread(os.path.join(data_dir,filename))


# 데이터 정규화

In [6]:
batch=batch.float()
batch/=255.0

In [7]:
n_channels=batch.shape[1]
for c in range(n_channels):
    mean=torch.mean(batch[:,c])
    std=torch.std(batch[:,c])
    batch[:,c]=(batch[:,c]-mean/std)

# 3차원 이미지: 용적 데이터

In [8]:
#특수 포맷 로딩
import imageio

dir_path='data/p1ch4/volumetric-dicom/2_LUNG'
vol_arr= imageio.volread(dir_path,'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%11/99 files (11.1%27/99 files (27.3%37/99 files (37.4%53/99 files (53.5%67/99 files (67.7%80/99 files (80.8%94/99 files (94.9%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 35/99  (35.468/99  (68.799/99  (100.0%)


(99, 512, 512)

In [9]:
vol=torch.from_numpy(vol_arr).float()
vol=torch.unsqueeze(vol,0)

vol.shape

torch.Size([1, 99, 512, 512])

In [10]:
torch.Size([1,99,512,512])

torch.Size([1, 99, 512, 512])

# 테이블 데이터 표현

In [11]:
import csv
import numpy as np
wine_path="data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy=np.loadtxt(wine_path,dtype=np.float32,delimiter=";",skiprows=1)

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [12]:
col_list=next(csv.reader(open(wine_path),delimiter=';'))

wineq_numpy.shape,col_list


((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [13]:
wineq=torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

## 점수 표현하기 (실수)

In [14]:
data=wineq[:,:-1] #모든 행과 마지막을 제외한 모든 열 선택
data,data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [21]:
target=wineq[:,-1].long()# 모든 행, 마지막 열 선택
target,target.shape

(tensor([6, 6, 6,  ..., 6, 7, 6]), torch.Size([4898]))

## 원핫 인코딩 scatter

In [22]:
target_onehot=torch.zeros(target.shape[0],10) 
target_onehot.scatter_(1, target.unsqueeze(1),1.0) # dim,index,src -> tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## 텐서 형태의 데이터 가공

In [23]:
data_mean=torch.mean(data,dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [24]:
data_var=torch.var(data,dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [27]:
data_normalized=(data-data_mean)/torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

## 임계값으로 구분

In [28]:
bad_indexes=target<=3 #
bad_indexes.shape,bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [29]:
bad_data=data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [33]:
bad_data=data[target<=3]
mid_data=data[(target>3)& (target<7)]
good_data=data[target>=7]

bad_mean=torch.mean(bad_data, dim=0)
mid_mean=torch.mean(mid_data, dim=0)
good_mean=torch.mean(good_data, dim=0)

for i,args in enumerate(zip(col_list,bad_mean,mid_mean,good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [34]:
total_sulfur_threshold=141.83 #이산화황(sulfeur dioxide)가 높으면 나쁜 와인임
total_sulfur_data=data[:,6] # 이산화황 총량에서 중앙점보다 낮은 인덱스 가져오기
predicted_indexes=torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()


(torch.Size([4898]), torch.bool, tensor(2727))

In [35]:
actual_indexes=target>5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [36]:
n_matches=torch.sum(actual_indexes& predicted_indexes).item()
n_predicted=torch.sum(predicted_indexes).item()
n_actual=torch.sum(actual_indexes).item()

In [37]:
n_matches, n_matches/n_predicted, n_matches/n_actual

(2018, 0.74000733406674, 0.6193984039287906)

# 시계열 데이터셋

In [39]:
bikes_numpy=np.loadtxt(
    "data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
    dtype=np.float32,
    delimiter=",",
    skiprows=1,
    converters={1:lambda x: float(x[8:10])}) #첫번째 열의 문자열으 숫자로 변환

bikes=torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [40]:
bikes.shape, bikes.stride() # 17520 시간, 17개 열

(torch.Size([17520, 17]), (17, 1))

In [41]:
daily_bikes=bikes.view(-1,24,bikes.shape[1]) # 저장공간을 바꾸지 않고 차원 수, 스트라이드 정보를 바꾼 새 텐서 넘김
daily_bikes.shape, daily_bikes.stride() # 730일 24시간 17열 -> 시간을 한칸 이동하려면 17열이동, 일자를 한칸 옮기렴녀 한행 *24 곱해야함


(torch.Size([730, 24, 17]), (408, 17, 1))

In [42]:
#730일(채널), 17개 값, 24시간 -> 텐서 전치
daily_bikes=daily_bikes.transpose(1,2)
daily_bikes.shape,daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

## 훈련 준비

In [43]:
# 날씨 상태 (1좋음 -> 4나쁨)
# 첫날의 데이터 
first_day=bikes[:24].long()
weather_onehot=torch.zeros(first_day.shape[0],4)
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [47]:
# 첫날 데이터 원핫 인코딩
weather_onehot.scatter_(
    dim=1,
    index=first_day[:,9].unsqueeze(1).long()-1, #인덱스가 0부터 시작하여 1 뻄
    value=1.0)


tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [48]:
torch.cat((bikes[:24],weather_onehot),1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [49]:
daily_weather_onehot=torch.zeros(daily_bikes.shape[0],4,daily_bikes.shape[2])

In [50]:
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [52]:
daily_weather_onehot.scatter_( #텐서 c차원에 원핫인코딩
    1,daily_bikes[:,9,:].long().unsqueeze(1)-1,1.0) # 9번째 열이 날씨
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [53]:
daily_bikes=torch.cat((daily_bikes,daily_weather_onehot),dim=1) # 병합
daily_bikes[:,9,:]=(daily_bikes[:,9,:]-1.0)/3.0


temp=daily_bikes[:,10,:] #10번째열인 온도 값(연속)을 0~1사이로 정규화함
temp_min=torch.min(temp)
temp_max=torch.max(temp)
daily_bikes[:,10,:]=((daily_bikes[:,10,:]-temp_min)/(temp_max-temp_min))


In [54]:
temp=daily_bikes[:,10,:]
daily_bikes[:,10,:]=((daily_bikes[:,10,:]-torch.mean(temp))/torch.std(temp))

# 텍스트 데이터

In [81]:
with open('data/p1ch4/jane-austen/1342-0.txt',encoding='utf8') as f:
    text=f.read()

In [82]:
lines=text.split('\n') #텍스트를 행으로 나눔
line=lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [83]:
letter_t=torch.zeros(len(line),128) # ASCII 제한 128로 하드코딩
letter_t.shape

torch.Size([70, 128])

In [84]:
# 문장단위 원핫인코딩
for i ,letter in enumerate(line.lower().strip()):
    letter_index=ord(letter) if ord(letter)<128 else 0 # 아스키코드 반환
    letter_t[i][letter_index]=1

In [94]:
# 단어 단위 원핫인코딩
# 소문자로 바꾸고 구두점 날리는 함수 생성
def clean_words(input_str):
    punctuation='.,;:"!?“"_-'
    word_list=input_str.lower().replace('\n',' ').split()
    word_list=[word.strip(punctuation) for word in word_list]
    return word_list

In [95]:
words_in_line=clean_words(line)
line,words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [96]:
word_list=sorted(set(clean_words(text)))
word2index_dict={word: i for (i,word) in enumerate(word_list)}

len(word2index_dict), word2index_dict['impossible']

(8267, 3848)

In [98]:
word_t=torch.zeros(len(words_in_line),len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index=word2index_dict[word]
    word_t[i][word_index]=1
    print('{:2} {:4} {}'.format(i,word_index,word))
print(word_t.shape)

 0 3848 impossible
 1 4928 mr
 2  899 bennet
 3 3848 impossible
 4 8049 when
 5 3759 i
 6  449 am
 7 5081 not
 8  249 acquainted
 9 8126 with
10 3637 him
torch.Size([11, 8267])
